In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai_tools import tool
import yfinance as yf


class Tools:

    @tool("One month stock price history")
    def stock_price(ticker):
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Stock news URLs")
    def stock_news(ticker):
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker):
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker):
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker):
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

In [3]:
from crewai import Agent


from crewai_tools import ScrapeWebsiteTool


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                ScrapeWebsiteTool(),
            ],
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )

In [4]:
from crewai import Task


class Tasks:

    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_market/stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="stock_market/technical_analysis.md",
        )

    def finacial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="stock_market/financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="stock_market/investment_recommendation.md",
        )

In [5]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.finacial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    verbose=2,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)

result = crew.kickoff(
    inputs=dict(
        company="Salesforce",
    ),
)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Gather and analyze the latest news and market sentiment surrounding the stock of Salesforce. Provide a summary of the news and any notable shifts in market sentiment.


> Entering new CrewAgentExecutor chain...
To effectively gather and analyze the latest news and market sentiment surrounding the stock of Salesforce, I need to first gather relevant data and insights from team members with expertise in research and analysis. This will involve delegating tasks and asking questions to gather comprehensive information about Salesforce's stock performance, recent news, and market sentiment. 

First, I will delegate the task of gathering the latest news about Salesforce to the Researcher.

Action: Delegate work to co-worker
Action Input: {"task": "Gather the latest news articles and reports about Salesforce, focusing on any recent developments or announcements that could impact its stock performance.", "context": "We need to under

2024-11-06 16:16:58,708 - 3492 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--e13ac607d285fe2f54f946e9691e434ce7078df603e8c8d6243f5411135dce37


Thought: I have gathered Salesforce's balance sheet and income statement data. Now, I will analyze the insider transactions to understand the sentiment of those closely associated with the company, which can provide insights into its stock valuation and growth potential.

Action: Get insider transactions  
Action Input: {"ticker": "CRM"}   

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





2024-11-06 16:17:00,400 - 3492 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--b6388fddf9e3930cc346a89299cfc27adc86f1651a170ab07c205e7984a979cb


Thought: To analyze Salesforce's stock valuation effectively, I will gather relevant financial data, including the balance sheet, income statement, and insider transactions. This will provide insights into its financial health and performance.

Action: Balance sheet  
Action Input: {"ticker": "CRM"}   

,2024-01-31,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Treasury Shares Number,64000000.0,28000000.0,,,
Ordinary Shares Number,971000000.0,981000000.0,989000000.0,919000000.0,
Share Issued,1035000000.0,1009000000.0,989000000.0,919000000.0,
Net Debt,954000000.0,3585000000.0,5132000000.0,,
Total Debt,12588000000.0,14088000000.0,13985000000.0,6281000000.0,
Tangible Book Value,5748000000.0,2666000000.0,1216000000.0,11061000000.0,
Invested Capital,69072000000.0,68960000000.0,68727000000.0,44166000000.0,
Working Capital,2443000000.0,504000000.0,1062000000.0,4161000000.0,
Net Tangible Assets,5748000000.0,2666000000.0,1216000000.0,11061000000.0,
Capital Lease Obligations,3162000000.0,3487000000

...

2024-11-06 16:17:14,375 - 3492 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--b6388fddf9e3930cc346a89299cfc27adc86f1651a170ab07c205e7984a979cb


Thought: To analyze Salesforce's stock valuation effectively, I will gather relevant financial data, including the balance sheet, income statement, and insider transactions. This will provide insights into its financial health and performance.

Action: Balance sheet  
Action Input: {"ticker": "CRM"}   

,2024-01-31,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Treasury Shares Number,64000000.0,28000000.0,,,
Ordinary Shares Number,971000000.0,981000000.0,989000000.0,919000000.0,
Share Issued,1035000000.0,1009000000.0,989000000.0,919000000.0,
Net Debt,954000000.0,3585000000.0,5132000000.0,,
Total Debt,12588000000.0,14088000000.0,13985000000.0,6281000000.0,
Tangible Book Value,5748000000.0,2666000000.0,1216000000.0,11061000000.0,
Invested Capital,69072000000.0,68960000000.0,68727000000.0,44166000000.0,
Working Capital,2443000000.0,504000000.0,1062000000.0,4161000000.0,
Net Tangible Assets,5748000000.0,2666000000.0,1216000000.0,11061000000.0,
Capital Lease Obligations,3162000000.0,3487000000

2024-11-06 16:17:16,885 - 3492 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--f170ad49df0ce7d0bc5736f71efd7bd4bc39dfadf4eb32881dc2803e17e6ed2e


Thought: I have gathered the balance sheet and income statement data for Salesforce. Now, I will analyze the insider transactions to gain insights into the sentiment of those closely associated with the company. This data will help assess the stock's valuation and growth potential.

Action: Get insider transactions  
Action Input: {"ticker": "CRM"}   

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I have already collected the balance sheet and income statement data for Salesforce, but I am unable to access the insider transactions data. I will analyze the information I have collected so far to derive insights into Salesforce's stock valuation and growth potential.

### Financial Analysis of Salesforce (CRM)

#### Balance Sheet Analysis (as of January 31, 2024):
- **Total Assets**: $99.82 billion
- **Total Liabilities**: $40.18 billion
- **Stockholders' Equity**: $59.65 billion
- **Net Debt**: $0.95 billion, indicating 

2024-11-06 16:18:47,620 - 3492 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--4ed1824ecd00bacf0df0f4752c332ac5883d58410dc0576a83588cbed3af53b2
2024-11-06 16:18:51,402 - 3492 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--79c5b3616060ecb4183537b7bda166b6791fdd555a311b1f29f8a5f22aeec453
2024-11-06 16:18:53,262 - 3492 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--79c5b3616060ecb4183537b7bda166b6791fdd555a311b1f29f8a5f22aeec453
